In [1]:
import lkml as looker
from pprint import pprint
from google.oauth2 import service_account
import pandas_gbq
from contextlib import redirect_stdout
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import pandas
import os
import json
import sys
import yaml

import warehouse_target
import config

import git

pd.options.mode.chained_assignment = None

credentials = config.service_account

warehouse_name = config.warehouse_name
lookml_project = config.project_name

sql = warehouse_target.warehouse_schema
explore_sql = warehouse_target.explore_sql

if warehouse_name == 'big_query':

    # Run a Standard SQL query with the project set explicitly
    project_id = lookml_project
    df = pandas.read_gbq(sql, dialect='standard', project_id=lookml_project, credentials=credentials)

    df['description'] = df['description'].fillna('not available')

    df1 = df[['table_name','column_name','data_type','description']]

    df1['data_type'] = df1['data_type'].str.replace('TIMESTAMP','timestamp')
    df1['data_type'] = df1['data_type'].str.replace('DATE','date')
    df1['data_type'] = df1['data_type'].str.replace('INT64','number')
    df1['data_type'] = df1['data_type'].str.replace('FLOAT64','number')
    df1['data_type'] = df1['data_type'].str.replace('NUMERIC','number')
    df1['data_type'] = df1['data_type'].str.replace('STRING','string')
    df1['data_type'] = df1['data_type'].str.replace('BOOL','yesno')

    df2 = df1

    explore_df = pandas.read_gbq(explore_sql, dialect='standard', project_id=lookml_project, credentials=credentials)
   

elif warehouse_name == 'snowflake': 

    url = URL(

        account = config.snowflake_account,
        user =  config.snowflake_user,
        schema =  config.snowflake_schema,
        database =  config.snowflake_database,
        password =  config.snowflake_password,
        warehouse= config.snowflake_warehouse,
        role =  config.snowflake_role

    )

    engine = create_engine(url)

    connection = engine.connect()

    query = '''
    select * from snowflake_sample_data.information_schema.columns;
    '''

    df = pd.read_sql(query, connection)
    
    df['description'] = df['comment'].fillna('not available')
    
    df1 = df.groupby(['table_name', 'column_name','data_type','description']).size().reset_index().rename(columns={0:'count'})

    df2 = df1[['table_name','column_name','data_type','description']]

    connection.close()
    engine.dispose()

    
df3 = {n: grp.loc[n].to_dict('index')
    
for n, grp in df2.set_index(['table_name', 'column_name','data_type','description']).groupby(level='table_name')}

d1 = df3

##

df4 = {n: grp.loc[n].to_dict('index')
    
for n, grp in explore_df.set_index(['parent_table_name','pk_table_name', 'pk_column_name','fk_table_name','fk_column_name']).groupby(level='parent_table_name')}

d2 = df4

##

def recur_dictify(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1: return frame.values[0][0]
        return frame.values.squeeze()
    grouped = frame.groupby(frame.columns[0])
    d = {k: recur_dictify(g.iloc[:,1:]) for k,g in grouped}
    return d

d5 = (recur_dictify(explore_df))

				

In [2]:
relations = []

for key,value in d2.items():

    for key,value in value.items():

        for value in value.items():

            relations.append(value)


In [3]:
relations = []

for key,value in d2.items():

    for key1,value in value.items():

        for key2,value in value.items():

            relations.append(key2)
            
            

In [118]:
relations

[('pk_row_count', 1490),
 ('fk_row_count', 1490),
 ('parent_row_count', 368775),
 ('true_relationship', 'one_to_one'),
 ('looker_relationship', 'many_to_one'),
 ('pk_row_count', 50),
 ('fk_row_count', 112810),
 ('parent_row_count', 368775),
 ('true_relationship', 'one_to_many'),
 ('looker_relationship', 'many_to_one'),
 ('pk_row_count', 50),
 ('fk_row_count', 6193),
 ('parent_row_count', 368775),
 ('true_relationship', 'one_to_many'),
 ('looker_relationship', 'many_to_one'),
 ('pk_row_count', 50),
 ('fk_row_count', 364813),
 ('parent_row_count', 368775),
 ('true_relationship', 'one_to_many'),
 ('looker_relationship', 'many_to_one'),
 ('pk_row_count', 244),
 ('fk_row_count', 244),
 ('parent_row_count', 368775),
 ('true_relationship', 'one_to_one'),
 ('looker_relationship', 'many_to_one'),
 ('pk_row_count', 244),
 ('fk_row_count', 50),
 ('parent_row_count', 368775),
 ('true_relationship', 'many_to_one'),
 ('looker_relationship', 'many_to_one'),
 ('pk_row_count', 244),
 ('fk_row_count', 1

In [156]:
for key,value in d2.items():

    for key2,value2 in value.items():

            for key2,value2 in value2.items():

                    elem_2 = [value2]

res = []

for key,value in d2.items():

    seq = []

    res.append({

        "explore": key,

        "view_name": key,

        "joins": seq

    })

    for key1,value in value.items():     

        elem = ({

            "joins": [
                {
                "sql_on": key1[0]+"."+key1[1]+" = "+ key1[2]+"."+key1[3],
                ##"relationship": 
                "name": key1[0],
                "relationship": elem_2
                }

            ]
            })

        seq.append(elem)

        for value2 in value.items():

                elem_2 = [value2]

                ##seq.append(elem_2)

In [157]:
res

[{'explore': 'sales_applications',
  'view_name': 'sales_applications',
  'joins': [{'joins': [{'sql_on': 'core_users.user_pk = core_user_mapping.user_fk',
      'name': 'core_users',
      'relationship': ['one_to_one']}]},
   {'joins': [{'sql_on': 'core_institutions.institution_pk = product_progression_plans.institution_fk',
      'name': 'core_institutions',
      'relationship': [('looker_relationship', 'many_to_one')]}]},
   {'joins': [{'sql_on': 'core_institutions.institution_pk = core_programs.institution_fk',
      'name': 'core_institutions',
      'relationship': [('looker_relationship', 'many_to_one')]}]},
   {'joins': [{'sql_on': 'core_institutions.institution_pk = product_study_plans.institution_fk',
      'name': 'core_institutions',
      'relationship': [('looker_relationship', 'many_to_one')]}]},
   {'joins': [{'sql_on': 'core_countries.country_pk = core_country_mapping.country_fk',
      'name': 'core_countries',
      'relationship': [('looker_relationship', 'many_to

In [80]:
df10 =  explore_relationship_df.set_index('pk_table_name').T.to_dict('looker_relationship')

/var/folders/qj/lbqph1rd0mvbl2g1l310_5nr0000gn/T/ipykernel_94805/1211856461.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  df10 =  explore_relationship_df.set_index('pk_table_name').T.to_dict('looker_relationship')
/var/folders/qj/lbqph1rd0mvbl2g1l310_5nr0000gn/T/ipykernel_94805/1211856461.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  df10 =  explore_relationship_df.set_index('pk_table_name').T.to_dict('looker_relationship')


In [101]:
dict(explore_relationship_df.values)


{'core_users': 'many_to_one',
 'core_institutions': 'one_to_one',
 'core_countries': 'many_to_one',
 'core_agents': 'many_to_one',
 'product_study_plans': 'one_to_one',
 'sales_applications': 'one_to_one',
 'sales_student': 'one_to_one',
 'core_sponsors': 'one_to_one'}

In [11]:
explore_relationship_df = explore_df[['pk_table_name','looker_relationship']]

In [12]:
dictionary = explore_relationship_df.to_dict()

In [13]:
dictionary_2 = explore_relationship_df.to_dict('list')

In [14]:
dictionary_2

{'pk_table_name': ['core_users',
  'core_institutions',
  'core_institutions',
  'core_institutions',
  'core_countries',
  'core_countries',
  'core_countries',
  'core_countries',
  'core_countries',
  'core_agents',
  'product_study_plans',
  'core_institutions',
  'sales_applications',
  'sales_applications',
  'sales_applications',
  'sales_student',
  'core_sponsors',
  'product_study_plans'],
 'looker_relationship': ['many_to_one',
  'many_to_one',
  'many_to_one',
  'many_to_one',
  'many_to_one',
  'many_to_one',
  'many_to_one',
  'many_to_one',
  'many_to_one',
  'many_to_one',
  'many_to_one',
  'one_to_one',
  'one_to_one',
  'one_to_one',
  'one_to_one',
  'one_to_one',
  'one_to_one',
  'one_to_one']}

In [133]:
dictionary

{'looker_relationship': {0: 'many_to_one',
  1: 'many_to_one',
  2: 'many_to_one',
  3: 'many_to_one',
  4: 'many_to_one',
  5: 'many_to_one',
  6: 'many_to_one',
  7: 'many_to_one',
  8: 'many_to_one',
  9: 'many_to_one',
  10: 'many_to_one',
  11: 'one_to_one',
  12: 'one_to_one',
  13: 'one_to_one',
  14: 'one_to_one',
  15: 'one_to_one',
  16: 'one_to_one',
  17: 'one_to_one'}}

In [150]:
for key, value in dictionary_2.items():


    relationship = value

In [154]:
    for key,value in dictionary_2.items():
                

        elem_2 = [value2]

In [155]:
elem_2

[('looker_relationship', 'one_to_one')]

In [119]:
test = dictionary.get('looker_relationship')

In [120]:
test

In [24]:
for key,value in d2.items():

    for key2,value2 in value.items():

            for key2,value2 in value2.items():

                    elem_2 = [value2]

res = [{"includes": "test/path"}]

for key,value in d2.items():

    seq = []

    res.append([{

        "explore": key,

        "view_name": key,

        "joins": seq

    }])

    for key1,value in value.items():     

        elem = ([{

            "joins": [
                {
                "sql_on": key1[0]+"."+key1[1]+" = "+ key1[2]+"."+key1[3],
                ##"relationship": 
                "name": key1[0],
                "relationship": elem_2
                }

            ]
            }])

        seq.append(elem)

        ##for value2 in value.items():

                ##elem_2 = [value2]

                ##seq.append(elem_2)

In [25]:
res

[{'includes': 'test/path'},
 [{'explore': 'sales_applications',
   'view_name': 'sales_applications',
   'joins': [[{'joins': [{'sql_on': 'core_users.user_pk = core_user_mapping.user_fk',
        'name': 'core_users',
        'relationship': ['one_to_one']}]}],
    [{'joins': [{'sql_on': 'core_institutions.institution_pk = product_progression_plans.institution_fk',
        'name': 'core_institutions',
        'relationship': ['one_to_one']}]}],
    [{'joins': [{'sql_on': 'core_institutions.institution_pk = core_programs.institution_fk',
        'name': 'core_institutions',
        'relationship': ['one_to_one']}]}],
    [{'joins': [{'sql_on': 'core_institutions.institution_pk = product_study_plans.institution_fk',
        'name': 'core_institutions',
        'relationship': ['one_to_one']}]}],
    [{'joins': [{'sql_on': 'core_countries.country_pk = core_country_mapping.country_fk',
        'name': 'core_countries',
        'relationship': ['one_to_one']}]}],
    [{'joins': [{'sql_on': 

In [8]:
for key,value in d2.items():

    for key2,value2 in value.items():

            for key2,value2 in value2.items():

                    elem_2 = [value2]

res = []

for key,value in d2.items():

    seq = []

    res.append({

        "explore": key,

        "view_name": key,

        "joins": seq

    })

    for key1,value in value.items():     

        elem = ({

            "joins": [
                {
                "sql_on": key1[0]+"."+key1[1]+" = "+ key1[2]+"."+key1[3],
                ##"relationship": 
                "name": key1[0],
                "relationship": elem_2
                }

            ]
            })

        seq.append(elem)

     ##   for value2 in value.items():

             ##   elem_2 = [value2]

                ##seq.append(elem_2)

In [10]:
d = {'a':5, 'b':6, 'c': 3}
>>> d2 = {'a':6, 'b':7, 'c': 3}
>>> for (k,v), (k2,v2) in zip(d.items(), d2.items()):
    print (k, v)
    print (k2, v2)

a 5
a 6
b 6
b 7
c 3
c 3


In [6]:
res

[{'explore': 'sales_applications',
  'view_name': 'sales_applications',
  'joins': [{'joins': [{'sql_on': 'core_users.user_pk = core_user_mapping.user_fk',
      'name': 'core_users',
      'relationship': ['one_to_one']}]},
   {'joins': [{'sql_on': 'core_institutions.institution_pk = product_progression_plans.institution_fk',
      'name': 'core_institutions',
      'relationship': ['one_to_one']}]},
   {'joins': [{'sql_on': 'core_institutions.institution_pk = core_programs.institution_fk',
      'name': 'core_institutions',
      'relationship': ['one_to_one']}]},
   {'joins': [{'sql_on': 'core_institutions.institution_pk = product_study_plans.institution_fk',
      'name': 'core_institutions',
      'relationship': ['one_to_one']}]},
   {'joins': [{'sql_on': 'core_countries.country_pk = core_country_mapping.country_fk',
      'name': 'core_countries',
      'relationship': ['one_to_one']}]},
   {'joins': [{'sql_on': 'core_countries.country_pk = core_institutions.country_fk',
      '